# Simple RNN with one LSTM Layer:

In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import random

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RNN

from numpy import array
from keras.models import load_model

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import platform
import time
import pathlib
import os
from keras.callbacks import ModelCheckpoint

# Data Preparation:

In [2]:
#read all the data:
items = pd.read_csv("dressipi_recsys2022/item_features.csv")
purchase =  pd.read_csv("dressipi_recsys2022/train_purchases.csv")
sessions =  pd.read_csv("dressipi_recsys2022/train_sessions.csv")


In [55]:
items_56 = items[items.feature_category_id == 56]
items_47 = items[items.feature_category_id == 47]

In [69]:
def get_item_features_simplified():
    items_features = {}
    items_unique = items.item_id.unique()
    for i in tqdm(range(len(items_unique))):
        features = [items_56.iat[i,2],items_47.iat[i,2]]
        items_features[ items_unique[i]] = features
    return items_features
item_features = get_item_features_simplified()

100%|██████████| 23691/23691 [00:01<00:00, 16113.23it/s]


In [71]:
import time
purchase['date'] = pd.to_datetime(purchase['date'])
sessions['date'] = pd.to_datetime(sessions['date'])
max_day = purchase['date'].max() #max date 31/05/2021 
min_day = max_day -datetime.timedelta(31)
cond = purchase['date'] >= min_day
new_purchases = purchase[cond]
top_seller = new_purchases['item_id'].value_counts()[0:100].index.to_list()

In [72]:
max_day = sessions['date'].max() #max date 31/05/2021 
testing_date = max_day -datetime.timedelta(31)
cond_test = sessions['date'] >= testing_date
cond_train = sessions['date'] < testing_date
cond_test_purchase = purchase.date >= testing_date
cond_train_purchase = purchase.date < testing_date
sessions_train = sessions[cond_train]
sessions_test = sessions[cond_test]
purchase_train = purchase[cond_train_purchase]
purchase_test = purchase[cond_test_purchase]

In [24]:
def sessions_list(sessions,purchase):
    antique_session = sessions.iat[0,0]
    list_sessions = []
    actual_session = []
    purchase_session = 0
    for i in tqdm(range(len(sessions))):
        new_session = sessions.iat[i,0]
        item = sessions.iat[i,1]
        if(new_session != antique_session):
            actual_session.append(purchase.iat[purchase_session,1])
            purchase_session += 1
            list_sessions.append(actual_session)
            actual_session = []
            antique_session = new_session
        actual_session.append(item)
    list_sessions.append(actual_session)
    return list_sessions

In [25]:
session_item = sessions_list(sessions,purchase)

100%|██████████| 4743820/4743820 [03:03<00:00, 25826.59it/s]


In [26]:
def training_sequences(session_item2):
    train_sequence = np.zeros((len(session_item),6))
    trobat = False
    for i,x in tqdm(enumerate(session_item2)):
        length = len(x)
        llista = x.copy()
        if(length-1 < 5):
            for j in range(5-length+1):
                llista.insert(0,0)
        if(length-1 > 5):
            for j in range(length-6):
                llista.pop(0)
        llista.pop(-1)                                    
        train_sequence[i][0] = llista[0]
        train_sequence[i][1] = llista[1]
        train_sequence[i][2] = llista[2]
        train_sequence[i][3] = llista[3]
        train_sequence[i][4] = llista[4]
        train_sequence[i][5] = x[len(x)-1]
    return train_sequence

sequence_training = training_sequences(session_item)

1000000it [00:03, 256990.14it/s]


In [27]:
X = np.array(sequence_training)[:,:5] #sequence to predict from
y = np.array(sequence_training)[:,5:] #next item in the sequence
X = X.astype(int)
y = y.astype(int)
training = np.array(sequence_training)

In [75]:
def get_item_features_simplified(n_features):
    items_features = np.zeros((items.item_id.nunique(),n_features))
    items_unique = items.item_id.unique()
    for i in tqdm(range(len(items_unique))):
        features = [items_56.iat[i,2],items_47.iat[i,2]]
        items_features[i] = features
    feature_training = np.zeros((X.shape[0],X.shape[1],n_features))
    for i in tqdm(range(X.shape[0])):
        features_X = []
        for j in range(X.shape[1]):
            feature_training[i,j] = item_feautres[]
            
    return items_features
item_features = get_item_features_simplified(2)

100%|██████████| 23691/23691 [00:00<00:00, 27035.10it/s]


# Network:

In [5]:
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

In [4]:
number_items = len(items.item_id.unique())
number_items += 1
number_items

23692

In [30]:
item_index = {}
index_item = {}
item_index[0] = 0
index_item[0] = 0
for i,x in enumerate(items.item_id.unique()):
    index_item[i+1] = x
    item_index[x] = i+1

In [31]:
def training_dataset(X2,y2):
    #first X.
    X_training = np.zeros((X2.shape[0],X2.shape[1]))
    y_training = np.zeros(y2.shape[0])
    
    for i in range(X2.shape[0]):
        for j in range(X2.shape[1]):
            X_training[i,j] = item_index[X2[i,j]]
    
    for i in range(y2.shape[0]):
        y_training[i] = item_index[y2[i][0]]
            
    return X_training, y_training
X_training, y_training = training_dataset(X,y)  
X_training = X_training.astype(int)
y_training = y_training.astype(int)

In [8]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    
    return model

In [6]:
#let's define a sparse categorical crossentropy to start!
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )

**Metrics**

In [7]:
#let's define a sparse categorical crossentropy to start!
from tensorflow.keras.utils import to_categorical
def top_5(y_true, y_pred):
    return  tf.keras.metrics.sparse_top_k_categorical_accuracy(
    y_true, y_pred, k=5
)
def top_50(y_true, y_pred):
    return  tf.keras.metrics.sparse_top_k_categorical_accuracy(
    y_true, y_pred, k=50
)
def top_100(y_true, y_pred):
    return  tf.keras.metrics.sparse_top_k_categorical_accuracy(
    y_true, y_pred, k=100
)

**Model Compilation**

In [20]:
tf.keras.backend.clear_session()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.104.129.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.104.129.122:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [40]:
number_items

23692

In [32]:
with strategy.scope():
  model = create_model(6, number_items)
  adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(
      optimizer=adam_optimizer,
      loss=loss,
      metrics= ["acc",top_5,top_100],
      steps_per_execution=32
  )
model.fit(X_training,y_training,epochs=100, batch_size=128, validation_split = 0.01, verbose=1,
          callbacks  = [EarlyStopping(monitor='val_acc', patience=5),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_acc', save_best_only=True)]
            )
model.save_weights('/tmp/best.h5', overwrite=True)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


7735/7735 [==============================] - 63s 8ms/step - loss: 8.4906 - acc: 0.0120 - top_5: 0.0334 - top_100: 0.1807 - val_loss: 7.8355 - val_acc: 0.0234 - val_top_5: 0.0626 - val_top_100: 0.2620
Epoch 2/100
7735/7735 [==============================] - 48s 6ms/step - loss: 7.5341 - acc: 0.0285 - top_5: 0.0797 - top_100: 0.3159 - val_loss: 7.2782 - val_acc: 0.0396 - val_top_5: 0.1078 - val_top_100: 0.3632
Epoch 3/100
7735/7735 [==============================] - 48s 6ms/step - loss: 7.1181 - acc: 0.0433 - top_5: 0.1142 - top_100: 0.3914 - val_loss: 7.0141 - val_acc: 0.0533 - val_top_5: 0.1346 - val_top_100: 0.4157
Epoch 4/100
7735/7735 [==============================] - 48s 6ms/step - loss: 6.8550 - acc: 0.0548 - top_5: 0.1398 - top_100: 0.4393 - val_loss: 6.8625 - val_acc: 0.0637 - val_top_5: 0.1531 - val_top_100: 0.4459
Epoch 5/100
7735/7735 [==============================] - 47s 6ms/step - loss: 6.6646 - acc: 0.0632 - top_5: 0.1577 - top_100: 0.4726 - val_loss: 6.7553 - val_acc: 0

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 10)             236920    
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 23692)             2392892   
                                                                 
Total params: 2,674,212
Trainable params: 2,674,212
Non-trainable params: 0
_________________________________________________________________
